<a href="https://colab.research.google.com/github/flanaganc04/My-own-Neural-Net/blob/main/Digit_Recognizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import packages
from google.colab import drive
import os
import datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

In [2]:
# Mount Google Drive and try not to get yelled at
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Import data
train_data = pd.read_csv('/content/drive/MyDrive/Digit Recognizer/mnist_test.csv')
test_data = pd.read_csv('/content/drive/MyDrive/Digit Recognizer/mnist_train.csv')
X_train = train_data.iloc[1:, 1:]  # Features
X_train_np = X_train.to_numpy() *(1/255)#
print(X_train)
Y_train = train_data.iloc[1:, 0:1]   # Target variable
Y_train_np = Y_train.to_numpy()
# print(Y_train)
X_test = test_data.iloc[1:, 2:-1]  # Features
X_test_np = X_test.to_numpy()
# print(X_test)
Y_test = test_data.iloc[1:, 0:1]   # Target variable
Y_test_np = Y_test.to_numpy()
# print(Y_test)

      1x1  1x2  1x3  1x4  1x5  1x6  1x7  1x8  1x9  1x10  ...  28x19  28x20  \
1       0    0    0    0    0    0    0    0    0     0  ...      0      0   
2       0    0    0    0    0    0    0    0    0     0  ...      0      0   
3       0    0    0    0    0    0    0    0    0     0  ...      0      0   
4       0    0    0    0    0    0    0    0    0     0  ...      0      0   
5       0    0    0    0    0    0    0    0    0     0  ...      0      0   
...   ...  ...  ...  ...  ...  ...  ...  ...  ...   ...  ...    ...    ...   
9995    0    0    0    0    0    0    0    0    0     0  ...      0      0   
9996    0    0    0    0    0    0    0    0    0     0  ...      0      0   
9997    0    0    0    0    0    0    0    0    0     0  ...      0      0   
9998    0    0    0    0    0    0    0    0    0     0  ...      0      0   
9999    0    0    0    0    0    0    0    0    0     0  ...      0      0   

      28x21  28x22  28x23  28x24  28x25  28x26  28x27  28x28  


In [4]:
# Make a Weight Matrix of random values between 0-1. The grayscale values are between 0-255. If this needs to be a larger value than we can change the random function's bounds.
neurons = 25
weightMatrix = np.random.random((neurons, X_train_np.shape[1]))
print(weightMatrix)

[[0.8276875  0.28711815 0.93842028 ... 0.84956109 0.70168498 0.50218887]
 [0.92275759 0.99996249 0.68464098 ... 0.20062093 0.05935814 0.92931623]
 [0.2704257  0.60401361 0.16640781 ... 0.71829605 0.52801913 0.09142769]
 ...
 [0.04561023 0.27363003 0.95126301 ... 0.30524895 0.99222281 0.97251671]
 [0.94209927 0.05800139 0.21563979 ... 0.543295   0.69584141 0.25695304]
 [0.29559804 0.13010959 0.83897832 ... 0.83671053 0.81579622 0.45059663]]


In [5]:
# Create a temporary matrix to hold some stuff, mainly, we are multiplying all the inputs by all of their respective weights
tempMatrix = X_train_np @ weightMatrix.T
print(tempMatrix)

[[50.04144168 58.29318174 60.1610555  ... 57.17724514 55.98277176
  52.84703017]
 [20.25361096 17.27398397 21.01549063 ... 18.34410952 17.74444738
  19.30962152]
 [74.34219161 75.06915851 75.060169   ... 70.8774124  73.88383069
  71.30537064]
 ...
 [56.30126406 60.35598103 59.11084119 ... 59.36135999 54.9358611
  59.58664846]
 [51.93255265 53.55436551 53.82900659 ... 52.74311442 49.19601268
  51.33916412]
 [87.18761901 85.55705865 87.45237904 ... 79.6013528  83.1721917
  80.60083745]]


In [6]:
neurons2 = 10
weightMatrix = np.random.random((neurons2,tempMatrix.shape[1]))
print(weightMatrix)

[[0.78274523 0.90483588 0.01654067 0.96012937 0.90183459 0.42164718
  0.01905216 0.27897031 0.18843531 0.03305557 0.9039034  0.66919755
  0.91568717 0.42971847 0.61141107 0.66242323 0.6061852  0.10762083
  0.57092751 0.92003095 0.43784007 0.39660716 0.17837756 0.37244272
  0.69576239]
 [0.6499061  0.92780336 0.59043535 0.73425251 0.2363331  0.45581229
  0.98472611 0.91665585 0.19845156 0.86844505 0.93762848 0.58119002
  0.23715956 0.16838129 0.85560072 0.71438025 0.51655418 0.97404398
  0.69423585 0.55988169 0.62783041 0.11782776 0.34521956 0.45720094
  0.20677741]
 [0.62820469 0.136823   0.5598373  0.68855392 0.49872054 0.56859843
  0.59242245 0.19688679 0.03788847 0.83830396 0.99257074 0.07792798
  0.75869198 0.31743064 0.77839187 0.78675158 0.55481119 0.81881757
  0.20099121 0.34193321 0.40828389 0.51822057 0.53044765 0.17280542
  0.98204466]
 [0.19223304 0.00763016 0.40242496 0.70895528 0.28563236 0.60288344
  0.9794947  0.0158011  0.10673914 0.19163725 0.55218585 0.12366692
  0.85

In [7]:
tempMatrix = tempMatrix@weightMatrix.T
print(tempMatrix)

[[ 734.42127607  828.01954762  731.36902012 ...  669.00222371
   828.24816539  749.9469828 ]
 [ 251.69775325  285.4215794   255.4478571  ...  233.17285084
   282.65096352  260.0162007 ]
 [ 949.94378111 1063.97688333  942.90256168 ...  866.49334189
  1064.12487765  965.79493204]
 ...
 [ 751.92939585  838.60545495  744.51987268 ...  683.73117652
   836.78383904  759.22185697]
 [ 676.1956046   755.26324074  671.59429174 ...  618.39075157
   754.3950258   687.09717313]
 [1079.20628243 1204.42931704 1068.76869336 ...  982.51586808
  1206.27462487 1094.92005537]]


In [8]:
biasMatrix = np.ones((1,10))
biasMatrix = np.array(biasMatrix)
outputMatrix = tempMatrix + biasMatrix
print(outputMatrix)

[[ 735.42127607  829.01954762  732.36902012 ...  670.00222371
   829.24816539  750.9469828 ]
 [ 252.69775325  286.4215794   256.4478571  ...  234.17285084
   283.65096352  261.0162007 ]
 [ 950.94378111 1064.97688333  943.90256168 ...  867.49334189
  1065.12487765  966.79493204]
 ...
 [ 752.92939585  839.60545495  745.51987268 ...  684.73117652
   837.78383904  760.22185697]
 [ 677.1956046   756.26324074  672.59429174 ...  619.39075157
   755.3950258   688.09717313]
 [1080.20628243 1205.42931704 1069.76869336 ...  983.51586808
  1207.27462487 1095.92005537]]


In [9]:
row_sums = np.sum(outputMatrix, axis=0)
print(row_sums)

[6799823.42942917 7613683.92115988 6757114.60832147 5202549.33549496
 6995678.83915188 6278938.66711239 5136576.88810114 6191459.37271365
 7597475.68602184 6892355.2079042 ]


In [10]:
rowEuler = np.exp(row_sums)
print(rowEuler)

[inf inf inf inf inf inf inf inf inf inf]


/tmp/ipython-input-3617419787.py:1: RuntimeWarning: overflow encountered in exp
  rowEuler = np.exp(row_sums)
